In [ ]:
import torch
import torch.nn as nn 
import numpy as np
import json
# import torch_xla.core.xla_model as xm
from torch.nn.modules.pooling import MaxPool2d

N = 11
inputc = 3
batch = 4

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

class CNN(nn.Module):
  def __init__(self,ic,oc,pad=1):
    super().__init__()
    self.conv=nn.Conv2d(
        ic,oc,3,stride=1,padding=pad,dilation=1,
        groups=1,bias=False,padding_mode='zeros'
    )
    self.bn = nn.BatchNorm2d(oc)
    # 批量归一化
  def forward(self,x):
    y = self.conv(x)
    y = self.bn(y)
    y = torch.relu(y)
    # relu激活函数
    return y

class RSN(nn.Module):
  def __init__(self,ic,oc):
      super().__init__()
      self.conv_net = nn.Sequential(
          CNN(ic,oc),
          CNN(oc,ic)
      )
  def forward(self,x):
    x = self.conv_net(x) + x
    return x

class outputhdv1(nn.Module):
  def __init__(self,oc,hmc):
    super().__init__()
    self.cnn=CNN(oc,hmc)
    self.pL = nn.Conv2d(hmc,1,1)
  def forward(self,h):
    x = self.cnn(h)
    x = self.pL(x)
    x = x.squeeze(1)
    return x

class outputhdv2(nn.Module):
  def __init__(self,oc,hmc):
    super().__init__()
    self.cnn=CNN(oc,hmc)
    self.vL = nn.Conv2d(hmc,1,1)
    self.vL2 = nn.Linear(N*N,128)
    self.vL3 = nn.Linear(128,1)
    self.vL4 = nn.Sigmoid()
  def forward(self,h):
    x = self.cnn(h)
    x = self.vL(x)
    x = x.squeeze(1)
    x = x.view(-1,N*N)

    x = self.vL2(x)
    x = torch.relu(x)
    x = self.vL3(x)
    
    x = self.vL4(x)
    x = x.squeeze(1)
    # z = torch.zeros((batch,2),dtype=float,device=device)
    # for i in range(batch):
    #   z[i] = self.vL2(x[i].reshape(-1))
    # print(x.device,x.shape)
    return x


class model(nn.Module):
  def __init__(self,b,f):
    super().__init__()
    self.name = "res"
    self.size = (b,f)
    self.iph = CNN(inputc,f*2,pad = 2)
    self.pool1 = nn.MaxPool2d(3,stride = 1,padding = 0)
    self.tru = nn.ModuleList()
    self.tru.append(CNN(f*2,f))
    for i in range(b):
      self.tru.append(RSN(f,f))
    self.opt1 = outputhdv1(f,f)
    self.opt2 = outputhdv2(f,f)

  def forward(self,x):
    if(x.shape[1]==2):
      x = torch.cat((x,torch.zeros(batch,inputc-2,N,N,device=device)),dim=1)
    # print(x.shape)
    # nn.exit()
    h = self.iph(x)
    h = self.pool1(h)
    for blk in self.tru :
      h = blk(h)
    h = self.opt2(h)
    return h

ModelDic = {
    "res":model
}

class bd:
  def __init__(self):
    self.board = np.zeros(shape=(2,N,N))
  def transpose(self):
    self.board = self.board.swapaxes(1,2)
  def play(self,x,y,color):
    self.board[color,y,x]=1
  def isL(self,x,y):
    if(self.board[0,y,x] or self.board[1,y,x]):
      return False
    return True

def loadM(filep):
  if(filep=='stochastic'):
    return 'stochastic'
  model = torch.load(filep,map_location=torch.device('cpu'))
  return model


Using cuda device


上面部分是模型的建模

下面部分是模型的训练

In [ ]:
# 写自定义dataset类
# import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import numpy as np
import copy
import random
N = 11
T = 1
Nt = 500
Ntrc = 10

class Mydataset(torch.utils.data.Dataset):
    def __init__(self,filep):
        
        file = open(filep,"r")
        rcm = file.readlines()
        L = len(rcm)
        i = 0
        self.mrc = []

        # L = min(L,25000)

        while i < L:
            board=bd()
            p = int(rcm[i])
            sm = rcm[i+1].split()
            k = 0
            win = float(rcm[i+2])
            if p>=50 and p<=170: 
              #去掉部分极端数据 
              while k < p:
                  x = int(sm[k])
                  y = int(sm[k+1])
                  o = ((k//2) & 1)
                  if (k>0.7*p and random.randint(0,99)>69):
                    # 学习中局策略
                      a = copy.deepcopy(board.board)
                      a = a.astype(np.float32)
                      # if o == 0:
                      #   c = np.array(0,dtype=float)
                      # else :
                      #   c = np.array(1,dtype=float)  
                      # if o == 1:
                      #     b = np.asarray([a[1],a[0]])
                      #     a = b.swapaxes(1,2)
                      #     c = np.array([0,1],dtype=float)
                      # a=np.insert(a,2,a[0]-a[1],axis=0)
                      # md我是智障吧我直接把胜者信息透露给他了
                      self.mrc.append((a,win))
                  board.play(x,y,o)
                  k += 2
            i += 3

    def __getitem__(self,index):
        return self.mrc[index]
    def __len__(self):
        return len(self.mrc)

def modeltest(model):
    model.eval()
    sumloss = 0
    for input,label in test_loader:
      input = input.to(device)
      label = label.to(device)
      if(label.size()[0]!=batch):
          break      
      with torch.no_grad():
        outputs = model(input)
      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, label)
      sumloss += loss.item()
    print(f"Test Loss is {sumloss/(test_size)*batch:.6f}")

if __name__ == '__main__':

    # train_data = Mydataset("TrainData.in2")
    # test_data = Mydataset("TestData.in")
    dataset = Mydataset("TrainData.in2")

    # train_size = len(train_data)
    # test_size = len(test_data)
    train_size = int(0.97 * len(dataset))
    test_size = len(dataset) - train_size

    train_data, test_data = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_data, batch_size = batch,shuffle = True)
    test_loader = DataLoader(test_data, batch_size= batch, shuffle = False)

    # mymod = loadM("/content/NR1/model_2_55000.pth")
    mymod = model(10,96)
    mymod = mymod.to(device)

    print(train_size//batch,test_size//batch)
    # nn.exit(0)
    print(mymod.parameters)
    # nn.exit()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mymod.parameters())
    epochn = 6
    
    for epoch in range(epochn):
        runningloss = 0.0
        for i,data in enumerate(train_loader,0):
            input,label = data
            # print(input,label)
            input = input.to(device)
            label = label.to(device)
            # print(input.device,label.device)

            if(label.size()[0]!=batch):
              break

            optimizer.zero_grad()
            output = mymod(input)
            # print(output.device,label.device)

            # print(output.shape,output.dtype)
            # print(label.shape,label.dtype)
            loss = criterion(output,label)

            loss.backward()
            optimizer.step()
            # nn.exit()
            runningloss += loss.item()
            if(i % Nt == 0 and i != 0):
                print('[%d,%5d] loss : %.8f' %
                        (epoch+1,i+1,runningloss/Nt))
                if(i % (Nt*Ntrc) == 0):
                    torch.save(mymod, f"NR2/model_{epoch}_{i}.pth")
                    modeltest(mymod)
                runningloss = 0.0
                
    print("Finished Training")

# 19069

68028 2104
<bound method Module.parameters of model(
  (iph): CNN(
    (conv): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), bias=False)
    (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool1): MaxPool2d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (tru): ModuleList(
    (0): CNN(
      (conv): Conv2d(192, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): RSN(
      (conv_net): Sequential(
        (0): CNN(
          (conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): CNN(
          (conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, a

In [ ]:
from torch.nn.modules.pooling import MaxPool2d
import torch
import torch.nn as nn 
import numpy as np
# 必要的一些库
import json
N = 11
inputc = 3
batch = 4

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

class CNN(nn.Module):
  def __init__(self,ic,oc,pad=1):
    super().__init__()
    self.conv=nn.Conv2d(
        ic,oc,3,stride=1,padding=pad,dilation=1,
        groups=1,bias=False,padding_mode='zeros'
    )
    self.bn = nn.BatchNorm2d(oc)
    # 批量归一化
  def forward(self,x):
    y = self.conv(x)
    y = self.bn(y)
    y = torch.relu(y)
    # relu激活函数
    return y

class RSN(nn.Module):
  def __init__(self,ic,oc):
      super().__init__()
      self.conv_net = nn.Sequential(
          CNN(ic,oc),
          CNN(oc,ic)
      )
  def forward(self,x):
    x = self.conv_net(x) + x
    return x

class outputhdv1(nn.Module):
  def __init__(self,oc,hmc):
    super().__init__()
    self.cnn=CNN(oc,hmc)
    self.pL = nn.Conv2d(hmc,1,1)
  def forward(self,h):
    x = self.cnn(h)
    x = self.pL(x)
    x = x.squeeze(1)
    # with torch.autograd.set_detect_anomaly(True):
    #   pl = torch.exp(x)
    #   for i in range(batch):
    #     pl[i] = pl[i] / pl[i].sum()
    # 自定义softmax
    # print(pl.shape)
    # print(pl)
    # nn.exit(0)
    return x

class model(nn.Module):
  def __init__(self,b,f):
    super().__init__()
    self.name = "res"
    self.size = (b,f)
    self.iph = CNN(inputc,f*2,pad = 2)
    self.pool1 = nn.MaxPool2d(3,stride = 1,padding = 0)
    self.tru = nn.ModuleList()
    self.tru.append(CNN(f*2,f))
    for i in range(b):
      self.tru.append(RSN(f,f))
    # self.tru.append(nn.MaxPool2d(3,stride = 1,padding = 1))
    # self.tru.append(CNN(f,f))
    # for i in range(b):
    #   self.tru.append(RSN(f,f))
    # self.pool2 = (nn.AvgPool2d(3,stride = 1,padding = 1))
    self.opt = outputhdv1(f,f)

  def forward(self,x):
    # if(x.shape[1]==2):
    #   x = torch.cat((x,torch.zeros(batch,1,N,N)),dim=1).to(device)
    # print(x.device)
    # print(x.shape)
    # print(x)
    # exit(0)
    h = self.iph(x)
    h = self.pool1(h)
    for blk in self.tru :
      h = blk(h)
    h = self.opt(h)
    return h

ModelDic = {
    "res":model
}

class bd:
  def __init__(self):
    self.board = np.zeros(shape=(2,N,N))
  def transpose(self):
    self.board = self.board.swapaxes(1,2)
  def play(self,x,y,color):
    self.board[color,y,x]=1
  def isL(self,x,y):
    if(self.board[0,y,x] or self.board[1,y,x]):
      return False
    return True

def loadM(filep):
  if(filep=='stochastic'):
    return 'stochastic'
  model = torch.load(filep,map_location=torch.device('cpu'))
  return model


In [ ]:
# 写自定义dataset类
# import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import numpy as np
import copy
import random
N = 11
T = 1
Nt = 500
Ntrc = 10

class Mydataset(torch.utils.data.Dataset):
    def __init__(self,filep):
        
        file = open(filep,"r")
        rcm = file.readlines()
        L = len(rcm)
        i = 0
        self.mrc = []

        while i < L:
            board=bd()
            p = int(rcm[i])
            sm = rcm[i+1].split()
            k = 0
            # print(i)
            win = int(rcm[i+2])
            if p>=50 and p<=170: 
              #去掉部分极端数据 
              while k < p:
                  x = int(sm[k])
                  y = int(sm[k+1])
                  o = ((k//2) & 1)
                  if o == win and (random.randint(0,9)>7 or (random.randint(0,9)>5 and k>0.7*p)):
                    # 学习中局策略
                      a = copy.deepcopy(board.board)
                      a = a.astype(np.float32)
                      tmp = np.zeros(shape=(N,N))
                      tmp[x][y] = 1
                      if o == 1:
                          b = np.asarray([a[1],a[0]])
                          a = b.swapaxes(1,2)
                          tmp = tmp.swapaxes(0,1)
                      a=np.insert(a,2,a[0]-a[1],axis=0)
                      self.mrc.append((a,copy.deepcopy(tmp)))
                  board.play(x,y,o)
                  k += 2

            i += 3

    def __getitem__(self,index):
        return self.mrc[index]
    def __len__(self):
        return len(self.mrc)

def modeltest(model):
    model.eval()
    sumloss = 0
    for input,label in test_loader:
      input = input.to(device)
      label = label.to(device)
      if(label.size()[0]!=batch):
          break      
      with torch.no_grad():
        outputs = model(input)
      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, label)
      sumloss += loss.item()
    print(f"Test Loss is {sumloss/(test_size)*batch:.6f}")

if __name__ == '__main__':

    train_data = Mydataset("TrainData.in2")
    test_data = Mydataset("TestData.in")

    train_size = len(train_data)
    test_size = len(test_data)
    # train_size = int(0.94 * len(dataset))
    # test_size = len(dataset) - train_size
    # train_data, test_data = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_data, batch_size = batch,shuffle = True)
    test_loader = DataLoader(test_data, batch_size= batch, shuffle = False)

    mymod = loadM("/content/model_0_105000.pth")
    # mymod = model(9,128)
    mymod = mymod.to(device)

    print(train_size//batch,test_size//batch)
    print(mymod.parameters)
    # nn.exit()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.ASGD(mymod.parameters(),lr=0.001)
    epochn = 6
    
    for epoch in range(epochn):
        runningloss = 0.0
        for i,data in enumerate(train_loader,0):
            input,label = data

            input = input.to(device)
            label = label.to(device)
            # print(input.device,label.device)

            if(label.size()[0]!=batch):
              break

            optimizer.zero_grad()
            output = mymod(input)
            loss = criterion(output,label)
            loss.backward()
            optimizer.step()
            
            runningloss += loss.item()
            if(i % Nt == 0 and i != 0):
                print('[%d,%5d] loss : %.8f' %
                        (epoch+1,i+1,runningloss/Nt))
                if(i % (Nt*Ntrc) == 0):
                    torch.save(mymod, f"NR4/model_{epoch}_{i}.pth")
                    modeltest(mymod)
                runningloss = 0.0
                
    print("Finished Training")

# 24000

几次挺失败的训练，参数对应是

6块，72深度或者42深度，加avgPool，最后误差率只能降低到0.083左右

当然每个都只跑到两个epoch...

好家伙，15块，10深度，只能跑到0.089左右的，还是跑了几乎3个epoch，每次都换优化方法跑

真的不如之前121通道4块的时候。。。能跑到0.69，简直是传奇，深度很重要啊（虽然数据集不一样，那时MCTS数据多）

6块128通道，有Maxpool带头，能跑到0.063，之后会过拟合。测试集降到0.062不下去了。

调整了优化方法（降低学习率），额外降低了0.005～0.006左右,差不多能到0.06以下了

谢谢，预测胜率完全预测不了，谢谢大家



# 强化学习